<a href="https://colab.research.google.com/github/mon-project-repos/ReviewGenerator/blob/master/ReviewGenerator_gensimWordVec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# set up session 
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)


import os
import tensorflow as tf
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ERROR: Not connected to a TPU runtime


In [0]:
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np
import re


import matplotlib.image  as mpimg
#import matplotlib.pyplot as plt
#%matplotlib ipympl
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])

import nltk
# nltk.download('stopwords')
#nltk.download('punkt')

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
percent_validation = 0.2
max_vocab_words = 20000
len_ngram = 20

#Load Data and Construct User and Product Docs

In [0]:
dta = pickle.load(open("gdrive/My Drive/dta_review.pkl",'rb'))
dta = dta.loc[dta['prod_total']>100]




prod_unique = dta.product_id.unique()
newprodid = pd.concat( [pd.Series(prod_unique) , pd.Series(range( len(prod_unique)) )], axis=1)
newprodid.columns = ['product_id','new_product_id']
dta = pd.merge(dta, newprodid, how='left', on='product_id')


class_unique = dta.text_class_id.unique()
newclassid = pd.concat([pd.Series(class_unique), pd.Series(range(len(class_unique)))], axis=1)
newclassid.columns = ['text_class_id', 'new_class_id']
dta = pd.merge(dta, newclassid, how='left', on='text_class_id')


reviewer_unique = dta.reviewer_id.unique()
newreviewerid = pd.concat([pd.Series(reviewer_unique),
                           pd.Series(range(len(reviewer_unique)))], axis=1)
newreviewerid.columns = ['reviewer_id', 'new_reviewer_id']
dta = pd.merge(dta, newreviewerid, how='left', on='reviewer_id')


review_unique = dta.review_id.unique()
newreviewid = pd.concat([pd.Series(review_unique),
                           pd.Series(range(len(review_unique)))], axis=1)
newreviewid.columns = ['review_id', 'new_review_id']
dta = pd.merge(dta, newreviewid, how='left', on='review_id')

#dta = dta.drop(columns=['id_x','id_y'])
#print(dta.head())
#print(dta.shape)
#print(dta.loc[dta['new_reviewer_id'==15],:].head())

n_users, n_products = len(dta.new_reviewer_id.unique()),  len(dta.new_product_id.unique())





# Pre process NLP

In [43]:
import gensim

min_word_occur = 5
word_vector_dim = 100
#from gensim.models import Word2Vec
word_list = [tf.keras.preprocessing.text.text_to_word_sequence(x) for x in dta['review_data'] ]
w2v = gensim.models.Word2Vec(word_list,
                            iter=10,
                            min_count = min_word_occur,
                            size= word_vector_dim,
                            workers = 4)
w2v.wv.vocab

{'i': <gensim.models.keyedvectors.Vocab at 0x7f4574688eb8>,
 'have': <gensim.models.keyedvectors.Vocab at 0x7f4574688ef0>,
 'tried': <gensim.models.keyedvectors.Vocab at 0x7f4574688f28>,
 'all': <gensim.models.keyedvectors.Vocab at 0x7f4574688f60>,
 'different': <gensim.models.keyedvectors.Vocab at 0x7f4574688f98>,
 'styles': <gensim.models.keyedvectors.Vocab at 0x7f4574688fd0>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7f4574691048>,
 'asics': <gensim.models.keyedvectors.Vocab at 0x7f4574691080>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7f45746910b8>,
 'running': <gensim.models.keyedvectors.Vocab at 0x7f45746910f0>,
 'shoes': <gensim.models.keyedvectors.Vocab at 0x7f4574691128>,
 'contend': <gensim.models.keyedvectors.Vocab at 0x7f4574691160>,
 'cumulus': <gensim.models.keyedvectors.Vocab at 0x7f4574691198>,
 'nimbus': <gensim.models.keyedvectors.Vocab at 0x7f45746911d0>,
 'gt': <gensim.models.keyedvectors.Vocab at 0x7f4574691208>,
 '1000': <gensim.models.keyedvectors.Vocab 

In [0]:

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_vocab_words+1,
                                                  lower=True, 
                                                  filters="",
                                                 oov_token="<OOV>",
                                                  char_level=False)

In [0]:

from gensim.models import Word2Vec
n_epochs= 10;
min_count = 1; #min_count, specifies the minimum amount of times that the word has to appear in the corpus before it is included in the vocabulary
EMBEDDING_DIM =100;
word2vec_model = Word2Vec(stemmed_review_data, 
                        iter=n_epochs, 
                        min_count=min_count, 
                        size=EMBEDDING_DIM, 
                          workers=4)

# append sequences (integer lists) to dta so we can use a groupby
sequences = [[word2vec_model.wv.vocab[stem_wd].index 
              for stem_wd in tokenreview] 
             for review in dta['review_data']]
dta['sequences'] = sequences
# #-----------

# first_validate = int(len(dta)*(1-percent_validation))
# print('Total',dta.shape, 'first validate:' ,first_validate)

# review_train = dta['review_data'].iloc[:first_validate]
# review_validate = dta['review_data'].iloc[first_validate: ]
# #corpus_train = (" end-of-review ").join( )
# #corpus_validation = (" end-of-review ").join( dta['review_data'].iloc[first_validate:])
# dta = None

In [0]:



corpus_train = (" endOfReview/n").join(review_train)
tokenizer.fit_on_texts(tf.keras.preprocessing.text.text_to_word_sequence(corpus_train))
total_words = len(tokenizer.word_index) + 1

word_freq = tokenizer.word_counts
word_idx = tokenizer.word_index
print(word_idx)
print(word_freq)

#print(len(tokenizer.word_index))
print(len(word_freq))

idx_word = {v:k for k,v in word_idx.items()}
print(idx_word)
print(len(idx_word))

In [26]:
#-----------------




# group documents by userId or productId, set to max num words and pad sequence
def joinAndPad(x, max_len ):
  doc = list(x['review_data']) + []
  #print(doc.__class.__, len(doc))
  #padded_doc = pad_sequences(doc, maxlen=max_len)
  return padded_doc


#list(dta.iloc[0:3]['review_data']) + []
prod_lens = dta.groupby('new_product_id').apply( lambda x:  len( list( x['review_data']) + []))

user_lens = dta.groupby('new_reviewer_id').apply( lambda x:  len( list(x['review_data']) +[]) )
print('prodDocs', 'max: ', max(prod_lens), 'min: ' ,min(prod_lens), 'avg:', np.mean(prod_lens), np.quantile(prod_lens,.75), np.quantile(prod_lens,.95))
print('userDocs', 'max: ', max(user_lens), 'min: ',min(user_lens),  'avg:', np.mean(user_lens), np.quantile(user_lens,.75), np.quantile(user_lens,.95))

max_len_prod = 5000
max_len_user = 25

docByProd = dta.groupby('new_product_id').apply(lambda x: joinAndPad(x ,max_len=max_len_prod) )
docByUser = dta.groupby('new_reviewer_id').apply(lambda x: joinAndPad(x, max_len=max_len_user))



# #dta3.head()
# print('prodDocs', len(docByProd))
# print('userDocs', len(docByUser))

prodDocs max:  6443 min:  110 avg: 927.0568181818181 1254.5 3431.95
userDocs max:  4 min:  1 avg: 1.0664880057520099 1.0 2.0
["I have tried all different styles of Asics for running shoes, Contend, Cumulus, Nimbus, Scram,  GT-1000, and GT-2000. All have been great shoes with the exception of the Scram which are technically trail runners but they were just to narrow in the toe box. These are a bit stiffer than other Asics but I hope that means they will hold up. They don't have as much cushion as the Cumulus or Nimbus and I will be going back to one of those for running. These are still good shoes, I just prefer the others.Edit:  These were bothering my feet while running. I bought gel inserts and they are so much better! The original inserts have no cushioning at all. ", "This is my 2nd pair of Asics, but NOT my favorite! I originally purchased Gel Contend 4's and fell in love with their comfort. I should've bought a 2nd pair of them but opted for a different color/style. Not as cushy 

ValueError: ignored

In [0]:
print(review_train[0])
print( tokenizer.texts_to_sequences([review_train[0]])[0])

In [0]:
def get_sequences(reviews,  len_ngram=20):
    sequences = []
    review_sentences = " ".join(reviews)
    review_tokens = tokenizer.texts_to_sequences([review_sentences])[0] #map each word to word_index 
    #for review in review_sentences:
        #review_words = review + " endOfReview/n"
        #tokenized_review = tokenizer.texts_to_sequences([review_words])[0] #map each word to word_index 
        #print(len(tokenized_review))
    #for i in range(1, min( [maxlen_ngram,len(review_words)])): # make ngrams
    for i in range(len_ngram, len(review_tokens)):
        sequences.append( review_tokens[i-len_ngram:i] )
    #if max_review_len is None:
    #    max_review_len = max([len(x) for x in sequences])
    #print(max_review_len)
    #sequences = np.array(pad_sequences(sequences,
    #                                     maxlen=maxlen_ngram,
    #                                     padding='pre')) 
    return np.array(sequences) #, max_review_len
        
def get_pred_lastword(sequences):
    predictors, last_word = sequences[:,:-1] , sequences[:,-1]
    #last_word = tf.keras.utils.to_categorical(last_word, num_classes=max_vocab_words+1)
    return (predictors, last_word )

        


In [0]:
train_sequences = get_sequences(review_train,len_ngram=len_ngram)
pred_train, lastword_train = get_pred_lastword(train_sequences)

validate_sequences = get_sequences(review_validate, len_ngram=len_ngram)
pred_validate, lastword_validate = get_pred_lastword(validate_sequences)

In [0]:
print('x=', pred_train.shape, 'y=', lastword_train.shape)
print(pred_train[:3])
print(lastword_train[:3])
print(pred_train.shape)
print(lastword_train.shape)
for m in range(24):
    print((" ").join([idx_word[i] if i>0 else "" for i in train_sequences[m]]))

print(review_train[0])